## GStreamer Basics - basic pipeline read buffer

### Resources:

Code adapted from:
https://paulbridger.com/posts/video-analytics-pytorch-pipeline/


Gstreamer References:
https://developer.download.nvidia.cn/embedded/L4T/r32_Release_v1.0/Docs/Accelerated_GStreamer_User_Guide.pdf

DeepStream:
https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvinfer.html

In [1]:
import os, sys
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst

import numpy as np
import torch, torchvision

preprocess = torchvision.transforms.ToTensor()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using: {device}')

Using: cuda


In [6]:
# Stream for my device - no display on screen  - extra nvvidconv needed since 
# gst buffer cannot read from memory otherwise. 
v7 = """
v4l2src device=/dev/video0 ! nvvidconv ! \
video/x-raw(memory:NVMM),framerate=(fraction)30/1,width=320,height=240 ! \
nvvidconv top=0 bottom=240 left = 90 right=320 ! \
video/x-raw(memory:NVMM),width=224,height=224 ! nvvidconv ! video/x-raw,format=RGBA ! \
fakesink name=webcam_stream
"""

v8 = """
v4l2src num-buffers=1000 device=/dev/video0 ! nvvidconv ! \
video/x-raw(memory:NVMM),framerate=(fraction)30/1,width=320,height=240 ! \
nvvidconv top=0 bottom=240 left = 90 right=320 ! \
video/x-raw(memory:NVMM),format=RGBA,width=224,height=224 ! nvvidconv ! video/x-raw ! \
queue ! tee name=t t. ! queue ! fakesink name=webcam_stream sync=true t. ! \
queue ! nvvidconv ! nvegltransform ! nveglglessink sync=true
"""

stream_string = v8

## Read from Frames into pytorch

In [7]:
pixel_bytes = 4

def on_frame_probe(pad, info):
    buffer = info.get_buffer()
    print(f'[{buffer.pts / Gst.SECOND:6.2f}]')

    image_tensor = buffer_to_image_tensor(buffer, pad.get_current_caps())
    image_batch = image_tensor.unsqueeze(0).to(device)
    
    with torch.no_grad():
        print('Inference Call Placeholder')
    return Gst.PadProbeReturn.OK

def buffer_to_image_tensor(buffer, caps):
    
    caps_structure = caps.get_structure(0)
    height = caps_structure.get_value('height')  
    width = caps_structure.get_value('width')

    is_mapped, map_info = buffer.map(Gst.MapFlags.READ)
    if is_mapped:
        try:
            image_array = np.ndarray(
                (height, width, pixel_bytes),
                dtype=np.uint8,
                buffer=map_info.data
            ).copy() # extend array lifetime beyond subsequent unmap
            return  torch.from_numpy(preprocess(image_array[:,:,:3]).copy) 
        
        finally:
            
            #Clean up the buffer mapping
            buffer.unmap(map_info)

In [8]:
stream_string = v8

# Instantiate Pipeline
Gst.init()

# Launch Pipeline
pipeline = Gst.parse_launch(f"{stream_string}")

# Probe read buffer and apply callback 'on_frame_probe'
pipeline.get_by_name('webcam_stream').get_static_pad('sink').add_probe(
    Gst.PadProbeType.BUFFER,
    on_frame_probe
)

# Set Pipeline to PLAYING
pipeline.set_state(Gst.State.PLAYING)

# Scan the pipeline bus for errors and other messages.
try:
    while True:
        msg = pipeline.get_bus().timed_pop_filtered(
            Gst.SECOND,
            Gst.MessageType.EOS | Gst.MessageType.ERROR
        )
        if msg:
            text = msg.get_structure().to_string() if msg.get_structure() else ''
            msg_type = Gst.message_type_get_name(msg.type)
            print(f'{msg.src.name}: [{msg_type}] {text}')
            break
            
# Break
except KeyboardInterrupt:
    print ('KeyboardInterrupt exception is caught')
    
# Write pipeline Graph
finally:
    print('Writing graph...')
    open(f'logs/logs.txt', 'w', encoding="utf8").write(
        Gst.debug_bin_to_dot_data(
            pipeline, Gst.DebugGraphDetails.ALL
        )
    )
    pipeline.set_state(Gst.State.NULL)
print('done.')

[  0.26]
Inference Call Placeholder
[  0.29]
Inference Call Placeholder
[  0.33]
Inference Call Placeholder
[  0.36]
Inference Call Placeholder
[  0.39]
Inference Call Placeholder
[  0.43]
Inference Call Placeholder
[  0.46]
Inference Call Placeholder
[  0.49]
Inference Call Placeholder
[  0.53]
Inference Call Placeholder
[  0.56]
Inference Call Placeholder
[  0.59]
Inference Call Placeholder
[  0.63]
Inference Call Placeholder
[  0.66]
Inference Call Placeholder
[  0.69]
Inference Call Placeholder
[  0.73]
Inference Call Placeholder
[  0.76]
Inference Call Placeholder
[  0.79]
Inference Call Placeholder
[  0.83]
Inference Call Placeholder
[  0.86]
Inference Call Placeholder
[  0.89]
Inference Call Placeholder
[  0.93]
Inference Call Placeholder
[  0.96]
Inference Call Placeholder
[  0.99]
Inference Call Placeholder
[  1.03]
Inference Call Placeholder
[  1.06]
Inference Call Placeholder
[  1.09]
Inference Call Placeholder
[  1.13]
Inference Call Placeholder
[  1.16]
Inference Call Plac

In [5]:
pipeline.set_state(Gst.State.NULL)

<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>